In [1]:
%pwd

'c:\\Users\\91889\\OneDrive\\Desktop\\Diamond-Price-Prediction\\research'

In [2]:
import os

os.chdir("../")

%pwd

'c:\\Users\\91889\\OneDrive\\Desktop\\Diamond-Price-Prediction'

### Config Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

# Data Transformation
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    target_column: str

### Configuration Manager

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema = self.schema.TARGET

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            target_column = schema.name
        )
        
        return data_transformation_config

### Components - Data Transformation

**Methods:**
- Outliers method
- Segregate method (Input and output)
- Train, Test and Split method.
- Transformation method (Standard Scalar, Ordinal Encoder)

`Note:` According to experiements, there are no missing values in the dataset. So, there is no need of Missing values method.

In [6]:
import os
from mlProject import logger

import pandas as pd
from sklearn.model_selection import train_test_split

# for numerical data
from sklearn.preprocessing import StandardScaler

# for categorical data
from sklearn.preprocessing import OrdinalEncoder

In [7]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    # outlier handling method
    def remove_outliers(self):
        df = pd.read_csv(self.config.data_path)

        # acc to experiments, these are the outliers
        df = df.loc[df['x'] > 3]
        df = df.loc[df['y'] < 15]
        df = df.loc[df['z'] < 10]
        df = df.loc[df['z'] > 2]

        return df
    
    # Segregate method
    def data_segregation(self):
        df = self.remove_outliers()

        # inputs (x)
        X = df.drop([self.config.target_column], axis=1)

        # output (y)
        y = df[[self.config.target_column]]

        return X, y
    
    # train test split method
    def get_train_test_split(self):
        X, y = self.data_segregation()

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

        return X_train, X_test, y_train, y_test
    
    # Transformation method
    def get_transformed(self):
        X_train, X_test, y_train, y_test = self.get_train_test_split()

        # diamond quality, low to high (ordinals)
        cut_categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
        color_categories = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
        clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
        
        # adjust Scaler and Encoder
        std_scaler = StandardScaler()
        ordinal_encoder = OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories],  
                                 handle_unknown="use_encoded_value",
                                 unknown_value=-1, 
                                 encoded_missing_value=-5)
        
        # train transformation
        X_train_num = X_train.select_dtypes(include=['int64', 'float64'])
        X_train_cat = X_train.select_dtypes(include=['object'])

        X_train_num_transformed = pd.DataFrame(std_scaler.fit_transform(X_train_num),
                                      columns = std_scaler.get_feature_names_out(),
                                      index = X_train_num.index)
        
        X_train_cat_transformed = pd.DataFrame(ordinal_encoder.fit_transform(X_train_cat),
                                      columns = ordinal_encoder.get_feature_names_out(),
                                      index = X_train_cat.index)
        
        # Concatenate
        X_train_transformed = pd.concat([X_train_num_transformed, X_train_cat_transformed], axis=1)

        # test transformation
        X_test_num = X_test.select_dtypes(include=['int64', 'float64'])
        X_test_cat = X_test.select_dtypes(include=['object'])

        X_test_num_transformed = pd.DataFrame(std_scaler.transform(X_test_num), 
                                   columns = std_scaler.get_feature_names_out(), 
                                   index = X_test_num.index)

        X_test_cat_transformed = pd.DataFrame(ordinal_encoder.transform(X_test_cat), 
                                   columns = ordinal_encoder.get_feature_names_out(), 
                                   index = X_test_cat.index)
        
        # Concatenate
        X_test_transformed = pd.concat([X_test_num_transformed, X_test_cat_transformed], axis=1)

        # save csv
        X_train_transformed.to_csv(os.path.join(self.config.root_dir, "X_train_transformed.csv"), index=False)
        y_train.to_csv(os.path.join(self.config.root_dir, "y_train.csv"), index=False)

        X_test_transformed.to_csv(os.path.join(self.config.root_dir, "X_test_transformed.csv"), index=False)
        y_test.to_csv(os.path.join(self.config.root_dir, "y_test.csv"), index=False)

        logger.info(f"X train transformed size: {X_train_transformed.shape}, y train size:{y_train.shape}")
        logger.info(f"X test transformed size: {X_test_transformed.shape}, y test size: {y_test.shape}")

### Pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)

    data_transformation.remove_outliers()
    data_transformation.data_segregation()
    data_transformation.get_train_test_split()
    data_transformation.get_transformed()

except Exception as e:
    raise e

[2024-10-20 15:40:52,144: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-20 15:40:52,150: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-20 15:40:52,154: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-20 15:40:52,156: INFO: common: created directory at: artifacts]
[2024-10-20 15:40:52,158: INFO: common: created directory at: artifacts/data_transformation]
[2024-10-20 15:40:53,136: INFO: 4146980770: X train transformed size: (40435, 9), y train size:(40435, 1)]
[2024-10-20 15:40:53,138: INFO: 4146980770: X test transformed size: (13479, 9), y test size: (13479, 1)]
